# Pre and post processing

[官网](https://github.com/facebookresearch/faiss/wiki/Pre--and-post-processing)

在某些情形下，需要对Index做前处理或后处理。

## ID映射

默认情况下，faiss会为每个输入的向量记录一个次序id，在使用中也可以为向量指定任意我们需要的id。  
部分index类型有`add_with_ids`方法，可以为每个向量对应一个64-bit的id，搜索的时候返回这个指定的id。

In [26]:
import sys
import faiss
import numpy as np 

d = 512
n_data = 2000

data = np.random.rand(n_data, d).astype('float32')
ids = np.arange(100000, 100000+n_data)

print(len(data), data[:5])
print(len(ids), ids[:5])

2000 [[0.24109639 0.39256188 0.53216225 ... 0.5932471  0.6245568  0.27635926]
 [0.30616745 0.48868874 0.36016256 ... 0.5516052  0.527553   0.8164353 ]
 [0.79772407 0.16738762 0.12488703 ... 0.38692498 0.14766863 0.21180402]
 [0.72823125 0.51616067 0.8114479  ... 0.56318784 0.99167514 0.30222702]
 [0.03934619 0.56336635 0.81954557 ... 0.93593997 0.7076944  0.5861631 ]]
2000 [100000 100001 100002 100003 100004]


In [27]:
k=6
nlist = 10
quantizer = faiss.IndexFlatIP(d)                                 # quantizer is IndexFlatIP
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2) # IndexIVFFlat is index

index.train(data)
index.add_with_ids(data, ids)                                    # add id to data

d, i = index.search(data[:5], k)

print(i)

[[100000 100691 101587 100578 101371]
 [100001 101699 100365 100444 100045]
 [100002 100134 101598 101126 101311]
 [100003 101042 100830 101933 100269]
 [100004 100552 100627 100231 100680]]


说明：
- 返回5个结果，返回`k`列
- 指定的`ids`不能是字符串，只能是整数。
- 返回的id是我们自己设定的，否则应该是0到4

但有些Index不支持`add_with_ids`，比如`IndexFlatL2`，需要与其他Index类型结合，将默认id映射到指定id，用`IndexIDMap`类实现。 
```python
index2 = faiss.IndexFlatL2(data.shape[1]) 
index2.add_with_ids(data, ids)  #报错
```

In [39]:
import sys
import faiss
import numpy as np 

d = 512
n_data = 2000

data = np.random.rand(n_data, d).astype('float32')
ids = np.arange(100000, 100000+n_data)

k=5
nlist = 10
index = faiss.IndexFlatL2(d)
index2 = faiss.IndexIDMap(index)  
index2.add_with_ids(data, ids)  #将index的id映射到index2的id,会维持一个映射表
index2.train(data)

D, I = index2.search(data[:5], k)

print(D)
print(I)

[[ 0.       73.10904  74.56877  75.1145   75.3489  ]
 [ 0.       71.581604 73.151184 74.22727  74.571014]
 [ 0.       70.626114 71.22928  73.958275 73.98425 ]
 [ 0.       70.35713  74.91534  76.67345  77.41719 ]
 [ 0.       71.314575 73.4188   74.1464   74.333115]]
[[100000 100286 100713 100191 100439]
 [100001 100453 100697 101900 101033]
 [100002 100186 100439 100853 101728]
 [100003 100592 101924 100743 100357]
 [100004 100083 100888 101196 100706]]


## 数据转换

有时候需要在索引之前转换数据。转换类继承了`VectorTransform`类，将输入向量转换为输出向量。  
1) 随机旋转,类名`RandomRotationMatri`,用以均衡向量中的元素，一般在`IndexPQ`和`IndexLSH`之前；  
2) PCA,类名`PCAMatrix`，降维；  
3) 改变维度，类名`RemapDimensionsTransform`，可以升高或降低向量维数

### 举例：PCA降维（通过IndexPreTransform）

输入向量是`2048`维，需要减少到`16byte`。

In [40]:
data = np.random.rand(n_data, 2048).astype('float32')
# the IndexIVFPQ will be in 256D not 2048
coarse_quantizer = faiss.IndexFlatL2 (256) 
sub_index = faiss.IndexIVFPQ (coarse_quantizer, 256, 16, 16, 8)
# PCA 2048->256
# 降维后随机旋转 (第四个参数)
pca_matrix = faiss.PCAMatrix (2048, 256, 0, True) 

#- the wrapping index
index = faiss.IndexPreTransform (pca_matrix, sub_index)

# will also train the PCA
index.train(data)  #数据需要是2048维
# PCA will be applied prior to addition
index.add(data)

### 举例：升维

有时需要在向量中插入0升高维度，一般是我们需要 
1) d是4的整数倍，有利于举例计算； 
2) d是M的整数倍。

In [41]:
d = 512
M = 8   #M是在维度方向上分割的子空间个数
d2 = int((d + M - 1) / M) * M
remapper = faiss.RemapDimensionsTransform (d, d2, True)
index_pq = faiss.IndexPQ(d2, M, 8)
index = faiss.IndexPreTransform (remapper, index_pq) #后续可以添加数据/索引

## 对搜索结果重新排序

当查询向量时，可以用真实距离值对结果进行重新排序。  
在下面的例子中，搜索阶段会首先选取4*10个结果，然后对这些结果计算真实距离值，再从中选取10个结果返回。`IndexRefineFlat`保存了全部的向量信息，内存开销不容小觑。

In [44]:
nbits_per_index = 4

data = np.random.rand(n_data, d).astype('float32')

q = faiss.IndexPQ(d, M, nbits_per_index)
rq = faiss.IndexRefineFlat(q)
rq.train(data)
rq.add(data)
rq.k_factor = 4

D, I = rq.search (data[:5], 10)

print(I)
print(D)

[[   0 1625 1553 1512 1938 1465   44 1603 1975  713]
 [   1 1724 1120 1611  100 1646 1871 1256 1337  589]
 [   2 1356  333  279 1947 1125  744  327  628 1030]
 [   3 1901 1403  433 1554 1191 1049  233   86  618]
 [   4 1268  238 1984  297 1306  186 1846 1763 1806]]
[[ 0.       74.520355 75.620316 77.98158  78.360535 78.40531  78.68231
  78.83919  78.90962  78.958984]
 [ 0.       74.23737  75.27939  75.91551  75.951324 76.46623  76.52034
  76.63651  76.81935  77.08807 ]
 [ 0.       75.48673  78.952614 79.02986  79.76219  79.892525 80.25382
  80.327576 80.40881  80.417046]
 [ 0.       76.52018  77.9581   78.53511  79.96349  80.31786  80.54482
  80.927    81.11404  82.119514]
 [ 0.       76.49896  77.389755 77.72101  77.84735  78.25365  78.93396
  79.61678  79.75316  79.880775]]


## 综合多个index返回的结果
当数据集分布在多个index中，需要在每个index中都执行搜索，然后使用IndexShards综合得到结果。同样也适用于index分布在不同的GPU的情况。